<a href="https://colab.research.google.com/github/lokeshsai-m/MTP/blob/main/Finetuning_textsummarization(ll-lstm).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [3]:
data = pd.read_csv('/content/dataset1.csv',encoding = 'unicode_escape')
data.drop_duplicates(keep='first')
# data.info()
# print(data.columns)
df = data[['ï»¿abstract', 'conclusion', 'tip', 'no']]

In [4]:
df = df.rename(columns={'no':'id', 'ï»¿abstract':'document', 'tip':'summary'})
# df = df.dropna()

In [5]:
df['document'] = df['document'].apply(lambda x: x.lower())
df['summary'] = df['summary'].apply(lambda x: x.lower())

In [6]:
!pip install transformers==4.17

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 28.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [7]:
!pip install datasets rouge-score  -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00


In [8]:
!pip install nltk

In [9]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
import datasets
from datasets import Dataset
from datasets import load_metric

In [11]:
import transformers
print(transformers.__version__)

4.17.0


In [12]:
from transformers import AutoTokenizer

model_checkpoint ='t5-small'

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

pad_on_right = tokenizer.padding_side == "right"

Downloading:   0%|          | 0.00/2.27k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

In [13]:
max_input_length = 1024
max_target_length = 128

In [14]:
df = df.sample(frac=1).reset_index(drop=True)
train = df[:40]
valid = df[41:46]
# test = df[47:]
# train.shape, valid.shape,test.shape

In [15]:
def preprocess_function(examples):
    inputs = ['summarize:' + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True,padding='max_length')

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [16]:
import torch
import datasets
from datasets import Dataset

train = Dataset.from_pandas(train)
valid = Dataset.from_pandas(valid)

In [17]:
tokenized_train = train.map(preprocess_function, batched=True)
tokenized_valid = valid.map(preprocess_function, batched=True)

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [18]:
from transformers import TFAutoModelForSeq2SeqLM
from torch import nn

In [19]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [20]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

In [21]:
batch_size = 16
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [22]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [23]:
import gc
gc.collect()

298

In [24]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("[INFO] training using {}".format(torch.cuda.get_device_name(0)))

[INFO] training using Tesla T4


In [25]:
%env WANDB_DISABLED=True

env: WANDB_DISABLED=True


In [26]:
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-tips",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=200,
    predict_with_generate=True,
    fp16=True
)

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [27]:
import tensorflow as tf

In [28]:
# Get the input size from the model's config
# input_size = model.config.d_model

# Create the additional LSTM layer
# model.additional_lstm = torch.nn.LSTM(input_size=input_size, hidden_size=256, num_layers=1, batch_first=True, bidirectional=False)

In [29]:
model.additional_lstm = tf.keras.layers.LSTM(units=256, return_sequences=True)

In [30]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp half precision backend


In [31]:
metric = load_metric("rouge")

<ipython-input-31-7154407f1f07>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [32]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: id, conclusion, document, summary. If id, conclusion, document, summary are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 40
  Num Epochs = 200
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 600


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.800539,18.935700,3.478300,15.613700,15.610700,19.000000
2,No log,2.774407,18.935700,3.478300,15.613700,15.610700,19.000000
3,No log,2.744412,18.935700,3.478300,15.613700,15.610700,19.000000
4,No log,2.728203,18.935700,3.478300,15.613700,15.610700,19.000000
5,No log,2.700861,17.731400,3.478300,15.613700,15.610700,19.000000
6,No log,2.674558,28.544300,14.247500,26.598600,26.598600,19.000000
7,No log,2.650865,28.544300,14.247500,26.598600,26.598600,19.000000
8,No log,2.628034,28.544300,14.247500,26.598600,26.598600,19.000000
9,No log,2.606897,28.725800,14.247500,26.780100,26.780100,19.000000
10,No log,2.587549,28.725800,14.247500,26.780100,26.780100,19.000000


The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: id, conclusion, document, summary. If id, conclusion, document, summary are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 5
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: id, conclusion, document, summary. If id, conclusion, document, summary are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 5
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: id, conclusion, document, summary. If id, conclusion, document, summary are not expected by `T5For

TrainOutput(global_step=600, training_loss=1.5387116241455079, metrics={'train_runtime': 524.916, 'train_samples_per_second': 15.241, 'train_steps_per_second': 1.143, 'total_flos': 2165468823552000.0, 'train_loss': 1.5387116241455079, 'epoch': 200.0})

In [33]:
test = df[44:]

In [34]:
print(test.shape)

(5, 4)


In [35]:
eval_dataset = Dataset.from_pandas(test)

In [36]:
eval_dataset = eval_dataset.map(
                preprocess_function,
                batched=True)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [37]:
predict_results = trainer.predict(
            eval_dataset,max_length=128, num_beams=3)

The following columns in the test set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: id, conclusion, document, summary. If id, conclusion, document, summary are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 5
  Batch size = 16


In [38]:
metrics = predict_results.metrics

In [39]:
metrics

{'test_loss': 2.1799466609954834,
 'test_rouge1': 23.7023,
 'test_rouge2': 6.7029,
 'test_rougeL': 19.232,
 'test_rougeLsum': 19.3928,
 'test_gen_len': 42.6,
 'test_runtime': 1.922,
 'test_samples_per_second': 2.601,
 'test_steps_per_second': 0.52}

In [40]:
if args.predict_with_generate:
    predictions = tokenizer.batch_decode(predict_results.predictions, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    predictions = [pred.strip() for pred in predictions]

In [41]:
predictions

['oxidative stress and carbonyl stress have essential mediatory roles in the development of diabetes and its related complications through increasing free radicals production and impairing antioxidant defense systems.',
 'ginger extract and gingerols enhanced glucose uptake inl6myotubes,byenhancingtranslocationofglut4tothesurfacemembraneandactivationofampk141through aca2+/calmodulin-dependent protein kinase pathway.',
 'bottle gourd pulp powder has therapeutic potential and may yield better results if consumed for longer duration.',
 'mushroom significantly reduced systolic and diastolic blood pressure (sbp, p0.01; dbp, p0.05); it also lowered both plasma glucose significantly (fpg & 2-hpg, p0.001).',
 'germination improves the mung beanâs nutritional value and its potential as a functional food.']

In [42]:
test['summary']

44    cucumber has protective effects in diabetes co...
45    there are positive effects of ginger in the me...
46    bottle gourd is high in dietary fiber and redu...
47    oyster mushrooms significantly reduce blood gl...
48    mustard oil can contribute to the improvement ...
Name: summary, dtype: object